# Домашнее задание 3, Лупашин Евгений, БПМИ184

## 0. Чтение данных

In [1]:
! pip install category-encoders
! pip install sweetviz
! pip install dtale
import pandas as pd
import numpy as np
import sweetviz as sv
import dtale
%matplotlib inline
from category_encoders.woe import WOEEncoder

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 1000)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

df_acc = pd.read_csv("12_accept.csv")
df_rej = pd.read_csv("12_reject.csv")

     |████████████████████████████████| 86 kB 345 kB/s eta 0:00:01
     |████████████████████████████████| 12.7 MB 41 kB/s eta 0:00:013
     |████████████████████████████████| 9.6 MB 543 kB/s eta 0:00:01
     |████████████████████████████████| 62 kB 62 kB/s  eta 0:00:01
     |████████████████████████████████| 1.2 MB 1.4 MB/s eta 0:00:01
     |████████████████████████████████| 27.7 MB 513 kB/s eta 0:00:01
     |████████████████████████████████| 45 kB 120 kB/s eta 0:00:01
     |████████████████████████████████| 642 kB 1.0 MB/s eta 0:00:01
     |████████████████████████████████| 870 kB 863 kB/s eta 0:00:01
     |████████████████████████████████| 79.9 MB 75 kB/s  eta 0:00:01     |██████████████████████████▉     | 67.1 MB 89 kB/s eta 0:02:24
     |████████████████████████████████| 47 kB 353 kB/s eta 0:00:01


     |████████████████████████████████| 209 kB 648 kB/s eta 0:00:01
     |████████████████████████████████| 357 kB 756 kB/s eta 0:00:01
  Created wheel for dash-colorscales: filename=dash_colorscales-0.0.4-py3-none-any.whl size=62589 sha256=f346f9676b31e725c85fd57b56250451a8c62ab573715e61a7a7f2878ae4fb93
  Stored in directory: /home/rinat/.cache/pip/wheels/ec/32/ec/dc8a1aa820182edf280a5e26c76fc9493ae29e196735708de2
  Created wheel for dash-daq: filename=dash_daq-0.5.0-py3-none-any.whl size=669716 sha256=d9d0d5ebc3030883ca71b7a16a7bdfe9b5fdb7842400f23307c3eea6b83da50a
  Stored in directory: /home/rinat/.cache/pip/wheels/53/73/ea/8a422aab5cab848d88653ed3087c1fa05553c3fbd67fb6ab75
  Created wheel for ppscore: filename=ppscore-1.2.0-py2.py3-none-any.whl size=13068 sha256=aa2b9c86f1151271e11c33c9f9f58163b57be434af2b09d168f50b381d092a28
  Stored in directory: /home/rinat/.cache/pip/wheels/66/5f/af/a0de66f8359588661c0b1239580f4788dba33a4a1e504ef682
Successfully built dash-colorscales dash-daq

In [2]:
# Первые строки заявок в accept
print('Размер accept:', df_acc.shape[0])
print('Размер reject:', df_rej.shape[0])
df_acc.head(10)

Размер accept: 32554
Размер reject: 10693


,target,INCOME_TYPE,EDUCATION_,CUSTOMER_CATEGORY_,income,DEPENDANT_NUMBER,IS_EMPL,COUNT_ACTIVE_CREDIT_NO_CC,CNT_MNTH_FROM_LAST_PMNT,age,experience,inquiry_14_day,inquiry_21_day,count_mnth_act_passport,UID
0,0,OTHER,elementary,NaN,140000.000,0.000,0,0.000,34.219,38.497,4.060,NaN,NaN,10.056,7
1,0,OTHER,elementary,NaN,50000.000,0.000,0,NaN,NaN,32.121,4.744,NaN,NaN,1.553,18
2,0,OTHER,elementary,Corporate,183461.000,0.000,1,0.000,1.187,44.063,0.512,1.000,1.000,2.649,23
3,0,OTHER,elementary,Corporate,183461.000,0.000,1,0.000,1.187,44.063,0.512,1.000,1.000,2.649,24
4,0,OTHER,elementary,Corporate,102984.000,0.000,1,NaN,NaN,34.884,6.120,NaN,NaN,1.758,34
5,0,OTHER,elementary,Corporate,127416.000,0.000,1,1.000,0.988,28.156,7.593,1.000,1.000,7.822,40
6,0,OTHER,elementary,NaN,123044.000,0.000,0,1.000,0.953,35.986,2.795,0.000,0.000,6.001,44
7,0,OTHER,beginner,Corporate,83623.000,0.000,1,1.000,0.218,28.966,3.716,0.000,0.000,8.421,56
8,0,2NDFL,elementary,NaN,34319.000,0.000,0,NaN,NaN,38.951,4.919,NaN,NaN,11.019,57
9,0,2NDFL,elementary,NaN,147900.000,0.000,0,NaN,NaN,37.890,8.060,NaN,NaN,12.288,59


## 1. Какая доля 1 в выборке "accept"?

In [3]:
df_acc[df_acc.target == 1].shape[0] / df_acc.shape[0]

0.021410579345088162

## 2. Статистики по переменным

Необходимо рассчитать для всех интервальных переменных следующее:
 - Доля пропущенных значений 
 - Медиана
 - Среднее
 - Среднеквадратическое отклонение
 - Есть ли аномальные значения, выбросы? 
 - Information Value
 
 
Необходимо рассчитать для всех категориальных переменных следующее:
 - Мода
 - Доля пропущенных значений
 - Information Value
 - Есть ли выбросы, аномальные значений
 
 
На всё, кроме выбросов и IV ответит отчёт, который генерируется в следующей клетке (`report.html`). По нажатию на признак видны его основные показатели. Выбросы далее тоже будем оценивать исходя из отчёта. Information Value для всех переменных и для всех датасетов посчитаем после обучения модели для разметки reject

In [4]:
# сразу удалим бесполезный столбец UID
df_acc.drop(columns='UID', inplace=True)
df_rej.drop(columns='UID', inplace=True)

report = sv.compare([df_acc, "AcceptDF"], [df_rej, "RejectDF"], "target")
report.show_html('report.html')

                                             |          | [  0%]   00:00 -> (? left)

Report report.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


Что насчёт выбросов?

Есть люди с доходом, превышающим 500000. Очевидно, таких людей мы будем относить к одной какой-то категории `богачи`, запомнили.

CNT_MNTH_FROM_LAST_PMNT ведёт себя довольно странно. Если я правильно понимаю, что это должно означать "количество месяцев с последнего платежа", то там не должно быть значений меньше 0. А они есть. И есть очень большие значения, больше 1000.

experience, тоже почему-то бывает отрицательным, а бывает и очень большим, но это нормально, вероятно.

count_mnth_act_passport, опять есть отрицательные значения

В категориальных признаках аномалий не обнаружено

## 4. Построить логистическую регрессию только на одобренных заявках с преобразованными переменными WoE. Какое значение GINI? F1 мера?

Сначала заполним пропуски на основе отчёта. income заполним нулями, возможно у людей, про которых мы не знаем их доход, он не очень велик. experience заполним нулём как самой популярной категорией, да и пропусков там мало. Пропуски в других категориях просто выделим в отдельную категорию

In [5]:
df_acc.income.fillna(value=0, inplace=True)
df_rej.income.fillna(value=0, inplace=True)

df_acc.experience.fillna(value=0, inplace=True)
df_rej.experience.fillna(value=0, inplace=True)

df_all = pd.concat([df_acc, df_rej])

df_all.income                  = pd.qcut(df_all.income, q=10, labels=None)
df_all.CNT_MNTH_FROM_LAST_PMNT = pd.qcut(df_all.CNT_MNTH_FROM_LAST_PMNT, q=10, labels=None)
df_all.age                     = pd.qcut(df_all.age, q=10, labels=None)
df_all.experience              = pd.qcut(df_all.experience, q=10, labels=None)
df_all.count_mnth_act_passport = pd.qcut(df_all.count_mnth_act_passport, q=10, labels=None)

dummies = pd.get_dummies(df_all, columns=df_all.columns.drop('target'), dummy_na=True)
df_acc = dummies.iloc[:df_acc.shape[0], :]
df_rej = dummies.iloc[df_rej.shape[0]:, :]

In [6]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, roc_auc_score

X = df_acc.copy()
y = X.target
X = X.drop('target', axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=228179, stratify=y)

Found ffmpeg: /opt/yandex/browser-beta/libffmpeg.so
	avcodec: 3868519
	avformat: 3868004
	avutil: 3736932
Ffmpeg version is OK! Let's use it.


In [7]:
%%time
WOE_encoder = WOEEncoder()
X_train = WOE_encoder.fit_transform(X_train, y_train)
X_test = WOE_encoder.transform(X_test)

CPU times: user 9.41 ms, sys: 173 µs, total: 9.58 ms
Wall time: 8.35 ms


In [8]:
model = RandomForestClassifier(max_features="log2", max_depth=10)

In [9]:
model.fit(X_train, y_train)

[8130:8130:0328/162506.862774:ERROR:variations_seed_processor.cc(251)] Trial from abt study=t2021 already created


RandomForestClassifier(max_depth=10, max_features='log2')

In [10]:
print('F1:', f1_score(model.predict(X_test), y_test))
roc_auc = roc_auc_score(y_test, model.predict_proba(X_test)[:, 1]) # roc_auc_score(model.predict(X_test), y_test)
Gini = 2 * roc_auc - 1
print('ROC AUC:', roc_auc)
print('Gini:', Gini)

F1: 0.08275862068965519
ROC AUC: 0.8041041742876884
Gini: 0.6082083485753769


## 5. Провести анализ Reject Inference. Какая доля отказанных заявок?

Теперь обучим на всей выборке и разметим reject по такому порогу, чтобы доля отказанных заявок была примерно равна оной в accept и была в районе 5-10%

In [21]:
df_rej

,target,INCOME_TYPE_2NDFL,INCOME_TYPE_OTHER,INCOME_TYPE_nan,EDUCATION__advanced,EDUCATION__beginner,EDUCATION__elementary,EDUCATION__high,EDUCATION__nan,CUSTOMER_CATEGORY__Corporate,CUSTOMER_CATEGORY__VIP,CUSTOMER_CATEGORY__nan,"income_(-0.001, 16317.0]","income_(16317.0, 21494.4]","income_(21494.4, 26100.0]","income_(26100.0, 30512.8]","income_(30512.8, 36378.0]","income_(36378.0, 43500.0]","income_(43500.0, 52080.0]","income_(52080.0, 69360.4]","income_(69360.4, 100000.0]","income_(100000.0, 11450378.0]",income_nan,DEPENDANT_NUMBER_0.0,DEPENDANT_NUMBER_1.0,DEPENDANT_NUMBER_2.0,DEPENDANT_NUMBER_3.0,DEPENDANT_NUMBER_4.0,DEPENDANT_NUMBER_nan,IS_EMPL_0.0,IS_EMPL_1.0,IS_EMPL_nan,COUNT_ACTIVE_CREDIT_NO_CC_0.0,COUNT_ACTIVE_CREDIT_NO_CC_1.0,COUNT_ACTIVE_CREDIT_NO_CC_2.0,COUNT_ACTIVE_CREDIT_NO_CC_3.0,COUNT_ACTIVE_CREDIT_NO_CC_4.0,COUNT_ACTIVE_CREDIT_NO_CC_5.0,COUNT_ACTIVE_CREDIT_NO_CC_6.0,COUNT_ACTIVE_CREDIT_NO_CC_7.0,COUNT_ACTIVE_CREDIT_NO_CC_8.0,COUNT_ACTIVE_CREDIT_NO_CC_nan,"CNT_MNTH_FROM_LAST_PMNT_(-31.074, 0.337]","CNT_MNTH_FROM_LAST_PMNT_(0.337, 0.468]","CNT_MNTH_FROM_LAST_PMNT_(0.468, 0.598]","CNT_MNTH_FROM_LAST_PMNT_(0.598, 0.735]","CNT_MNTH_FROM_LAST_PMNT_(0.735, 0.923]","CNT_MNTH_FROM_LAST_PMNT_(0.923, 1.119]","CNT_MNTH_FROM_LAST_PMNT_(1.119, 1.826]","CNT_MNTH_FROM_LAST_PMNT_(1.826, 5.587]",...,"age_(23.912, 24.683]","age_(24.683, 26.31]","age_(26.31, 28.741]","age_(28.741, 31.164]","age_(31.164, 34.053]","age_(34.053, 37.529]","age_(37.529, 41.759]","age_(41.759, 47.579]","age_(47.579, 58.938]",age_nan,"experience_(-0.001, 0.496]","experience_(0.496, 0.745]","experience_(0.745, 1.173]","experience_(1.173, 1.709]","experience_(1.709, 2.4]","experience_(2.4, 3.378]","experience_(3.378, 4.637]","experience_(4.637, 6.29]","experience_(6.29, 9.866]","experience_(9.866, 101.192]",experience_nan,inquiry_14_day_0.0,inquiry_14_day_1.0,inquiry_14_day_2.0,inquiry_14_day_3.0,inquiry_14_day_4.0,inquiry_14_day_5.0,inquiry_14_day_6.0,inquiry_14_day_nan,inquiry_21_day_0.0,inquiry_21_day_1.0,inquiry_21_day_2.0,inquiry_21_day_3.0,inquiry_21_day_4.0,inquiry_21_day_5.0,inquiry_21_day_6.0,inquiry_21_day_7.0,inquiry_21_day_8.0,inquiry_21_day_nan,"count_mnth_act_passport_(-0.00646, 1.991]","count_mnth_act_passport_(1.991, 3.277]","count_mnth_act_passport_(3.277, 4.008]","count_mnth_act_passport_(4.008, 4.683]","count_mnth_act_passport_(4.683, 5.874]","count_mnth_act_passport_(5.874, 7.395]","count_mnth_act_passport_(7.395, 8.813]","count_mnth_act_passport_(8.813, 9.784]","count_mnth_act_passport_(9.784, 10.77]","count_mnth_act_passport_(10.77, 43.099]",count_mnth_act_passport_nan
10693,0.000,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
10694,0.000,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
10695,0.000,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
10696,0.000,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0
10697,0.000,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,..

In [26]:
df_rej

,target,INCOME_TYPE_2NDFL,INCOME_TYPE_OTHER,INCOME_TYPE_nan,EDUCATION__advanced,EDUCATION__beginner,EDUCATION__elementary,EDUCATION__high,EDUCATION__nan,CUSTOMER_CATEGORY__Corporate,CUSTOMER_CATEGORY__VIP,CUSTOMER_CATEGORY__nan,"income_(-0.001, 16317.0]","income_(16317.0, 21494.4]","income_(21494.4, 26100.0]","income_(26100.0, 30512.8]","income_(30512.8, 36378.0]","income_(36378.0, 43500.0]","income_(43500.0, 52080.0]","income_(52080.0, 69360.4]","income_(69360.4, 100000.0]","income_(100000.0, 11450378.0]",income_nan,DEPENDANT_NUMBER_0.0,DEPENDANT_NUMBER_1.0,DEPENDANT_NUMBER_2.0,DEPENDANT_NUMBER_3.0,DEPENDANT_NUMBER_4.0,DEPENDANT_NUMBER_nan,IS_EMPL_0.0,IS_EMPL_1.0,IS_EMPL_nan,COUNT_ACTIVE_CREDIT_NO_CC_0.0,COUNT_ACTIVE_CREDIT_NO_CC_1.0,COUNT_ACTIVE_CREDIT_NO_CC_2.0,COUNT_ACTIVE_CREDIT_NO_CC_3.0,COUNT_ACTIVE_CREDIT_NO_CC_4.0,COUNT_ACTIVE_CREDIT_NO_CC_5.0,COUNT_ACTIVE_CREDIT_NO_CC_6.0,COUNT_ACTIVE_CREDIT_NO_CC_7.0,COUNT_ACTIVE_CREDIT_NO_CC_8.0,COUNT_ACTIVE_CREDIT_NO_CC_nan,"CNT_MNTH_FROM_LAST_PMNT_(-31.074, 0.337]","CNT_MNTH_FROM_LAST_PMNT_(0.337, 0.468]","CNT_MNTH_FROM_LAST_PMNT_(0.468, 0.598]","CNT_MNTH_FROM_LAST_PMNT_(0.598, 0.735]","CNT_MNTH_FROM_LAST_PMNT_(0.735, 0.923]","CNT_MNTH_FROM_LAST_PMNT_(0.923, 1.119]","CNT_MNTH_FROM_LAST_PMNT_(1.119, 1.826]","CNT_MNTH_FROM_LAST_PMNT_(1.826, 5.587]",...,"age_(23.912, 24.683]","age_(24.683, 26.31]","age_(26.31, 28.741]","age_(28.741, 31.164]","age_(31.164, 34.053]","age_(34.053, 37.529]","age_(37.529, 41.759]","age_(41.759, 47.579]","age_(47.579, 58.938]",age_nan,"experience_(-0.001, 0.496]","experience_(0.496, 0.745]","experience_(0.745, 1.173]","experience_(1.173, 1.709]","experience_(1.709, 2.4]","experience_(2.4, 3.378]","experience_(3.378, 4.637]","experience_(4.637, 6.29]","experience_(6.29, 9.866]","experience_(9.866, 101.192]",experience_nan,inquiry_14_day_0.0,inquiry_14_day_1.0,inquiry_14_day_2.0,inquiry_14_day_3.0,inquiry_14_day_4.0,inquiry_14_day_5.0,inquiry_14_day_6.0,inquiry_14_day_nan,inquiry_21_day_0.0,inquiry_21_day_1.0,inquiry_21_day_2.0,inquiry_21_day_3.0,inquiry_21_day_4.0,inquiry_21_day_5.0,inquiry_21_day_6.0,inquiry_21_day_7.0,inquiry_21_day_8.0,inquiry_21_day_nan,"count_mnth_act_passport_(-0.00646, 1.991]","count_mnth_act_passport_(1.991, 3.277]","count_mnth_act_passport_(3.277, 4.008]","count_mnth_act_passport_(4.008, 4.683]","count_mnth_act_passport_(4.683, 5.874]","count_mnth_act_passport_(5.874, 7.395]","count_mnth_act_passport_(7.395, 8.813]","count_mnth_act_passport_(8.813, 9.784]","count_mnth_act_passport_(9.784, 10.77]","count_mnth_act_passport_(10.77, 43.099]",count_mnth_act_passport_nan
10693,0.000,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
10694,0.000,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
10695,0.000,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
10696,0.000,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0
10697,0.000,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,..

In [11]:
model.fit(X, y)
df_pred = df_rej.copy().drop('target', axis=1)
df_pred['target'] = model.predict_proba(df_pred)[:, 1]
df_pred['target'] = np.where(df_pred['target'] > 0.06, 1, 0)

In [12]:
print('доля отказанных заявок в reject:', df_pred[df_pred.target == 1].shape[0] / df_pred[df_pred.target == 0].shape[0])

доля отказанных заявок в reject: 0.027426226921256115


In [13]:
print('доля отказанных заявок в accept:', df_acc[df_acc.target == 1].shape[0] / df_acc[df_acc.target == 0].shape[0])

доля отказанных заявок в accept: 0.021879021879021878


## 2 и 3 (продолжение)
Наконец после разметки данных посчитаем WoE и Information Value для всех переменных и для всех датасетов

In [14]:
# Полезные функции

# source: https://gist.github.com/dradecic/52d8b2b2213dd3d46f4b75f85c1183f2
def calculate_woe_iv(dataset, feature, target, iv=False, sort=False):
    lst = []
    for i in range(dataset[feature].nunique()):
        val = list(dataset[feature].unique())[i]
        lst.append({
            'Value': val,
            'All': dataset[dataset[feature] == val].count()[feature],
            'Good': dataset[(dataset[feature] == val) & (dataset[target] == 0)].count()[feature],
            'Bad': dataset[(dataset[feature] == val) & (dataset[target] == 1)].count()[feature]
        })
        
    dset = pd.DataFrame(lst)
    dset['Distr_Good'] = dset['Good'] / dset['Good'].sum()
    dset['Distr_Bad'] = dset['Bad'] / dset['Bad'].sum()
    dset['WoE'] = np.log(dset['Distr_Good'] / dset['Distr_Bad'])
    dset = dset.replace({'WoE': {np.inf: 0, -np.inf: 0}})
    dset['IV'] = (dset['Distr_Good'] - dset['Distr_Bad']) * dset['WoE']
    if sort:
        dset = dset.sort_values(by='WoE')
        
    if iv:
        iv = dset['IV'].sum()
        return dset, iv
    else:
        return dset

# source: https://www.kaggle.com/puremath86/iv-woe-starter-for-python
def woe_iv(dataframe, feature, target='target'):
    return (pd.crosstab(dataframe[feature], dataframe[target], normalize='columns')
        .assign(WoE = lambda df: np.log(df[1] / df[0])).replace({np.inf: 0, -np.inf: 0})
        .assign(IV = lambda df: np.nansum(df.WoE * (df[1] - df[0])))
    )

In [15]:
df_all_pred = df_acc.append(df_pred)
for col in list(df_all_pred.columns)[1:]:
    try:
        print(woe_iv(df_all_pred[cols], col), "\n")
    except:
        pass

[8130:8130:0328/162515.403052:ERROR:isolated_origin_util.cc(74)] Ignoring port number in isolated origin: chrome://custo


## 6. Построить логистическую регрессию на всех заявках с преобразованными переменными WoE

In [25]:
reject

NameError: name 'reject' is not defined

In [16]:
X_train, X_test, y_train, y_test = train_test_split(df_all_pred.drop('target', axis=1),
                                                    df_all_pred.target,
                                                    test_size=0.2,
                                                    random_state=4533357)

Opening in existing browser session.


In [17]:
%%time
WOE_encoder = WOEEncoder()
X_train = WOE_encoder.fit_transform(X_train, y_train)
X_test = WOE_encoder.transform(X_test)

CPU times: user 6.51 ms, sys: 0 ns, total: 6.51 ms
Wall time: 6.21 ms


In [18]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(max_iter=10000)

In [19]:
model.fit(X_train, y_train)

libva error: /usr/lib/x86_64-linux-gnu/dri/iHD_drv_video.so init failed
[8222:8222:0100/000000.123549:ERROR:sandbox_linux.cc(376)] InitializeSandbox() called with multiple threads in process gpu-process.


LogisticRegression(max_iter=10000)

In [20]:
print('F1:', f1_score(model.predict(X_test), y_test))
roc_auc = roc_auc_score(y_test, model.predict_proba(X_test)[:, 1])
Gini = 2 * roc_auc - 1
print('ROC AUC:', roc_auc)
print('Gini:', Gini)

F1: 0.1586402266288952
ROC AUC: 0.8827210598832539
Gini: 0.7654421197665078


## 7. Какую модель вы рекомендуете для внедрения в продуктивную среду? Дать развернутое пояснение

Вторая модель получилась лучше по всем использованным метрикам, конечно, среди этих двух я порекомендую её. В целом можно было подойти более подробно к изучению выбросов, разбиению переменные на бины и последующему объединению похожих бинов, подбору гиперпараметров моделей и много чему ещё, но модель и так получилась достаточно хорошей, на мой взгляд